In [1]:
import bs4 as bs 
import re
import pandas as pd
import numpy as np

'''
1. load html in code
2. tranform to soup
3. strip to only body

4. loop through and grab only:
    
5. 


Output: text file 

'''

'\n1. load html in code\n2. tranform to soup\n3. strip to only body\n\n4. loop through and grab only:\n    \n5. \n\n\nOutput: text file \n\n'

In [2]:
file_path = "notes/Ego Is the Enemy - Notebook.html"

In [3]:
f1 = open(file_path, 'r', encoding="utf-8").read()

In [4]:
soup = bs.BeautifulSoup(f1, features='html.parser') 

In [5]:
soup

<!DOCTYPE html PUBLIC
"-//W3C//DTD XHTML 1.0 Strict//EN"
"XHTML1-s.dtd" >

<html lang="en" xml:lang="en" xmlns="http://www.w3.org/TR/1999/REC-html-in-xml">
<head>
<meta charset="utf-8"/>
<style>
        .bodyContainer {
            font-family: Arial, Helvetica, sans-serif;
            text-align: center;
            padding-left: 32px;
            padding-right: 32px;
        }
        
        .notebookFor {
            font-size: 18px;
            font-weight: 700;
            text-align: center;
            color: rgb(119, 119, 119);
            margin: 24px 0px 0px;
            padding: 0px;
        }
        
        .bookTitle {
            font-size: 24px;
            font-weight: 700;
            text-align: center;
            color: #333333;
            margin-top: 22px;
            padding: 0px;
        }
        
        .authors {
            font-size: 18px;
            font-weight: 700;
            text-align: center;
            color: rgb(119, 119, 119);
            m

In [6]:
'''
get_book_details(soup)
returns metadata of the book

{
'Title': , 
'Authors': , 
'Citations':
}



'''

def get_book_details(soup):
    
    book_details = {}
    
    try:
        book_details['Title'] = soup.find('div', {'class': 'bookTitle'}).text.strip()
        book_details['Authors'] = soup.find('div', {'class': 'authors'}).text.strip()
        book_details['Citations']= soup.find('div', {'class': 'citation'}).text.strip()
    except:
        pass
    
    return book_details
    

In [7]:
# a list of all divs
all_divs = soup.findAll('div')
all_divs[5].text

'\n    THE PAINFUL PROLOGUE\n'

In [8]:
# find_next() returns next item on the list
all_divs[6].find_next()

<span class="highlight_orange">orange</span>

In [9]:
test = all_divs[6].text.strip()
test

'Highlight(orange) - Page xxiv · Location 185'

In [10]:
div_class = []

for item in all_divs:
    div_class.append(item['class'][0])

In [11]:
#unique classes:
#We care about noteHeading, sectionHeading & noteText
set(div_class)

{'authors',
 'bodyContainer',
 'bookTitle',
 'citation',
 'noteHeading',
 'noteText',
 'notebookFor',
 'sectionHeading'}

In [12]:
type_pattern = r'(.*)-'
page_pattern = r'Page (.+) ·'
location_pattern = r'Location (\d+)'

In [13]:
def get_page_location(div_soup):
    
    '''
    Use when div_soup[class] = noteHeading
    
    '''
    
    details = {}  
    
    try:
        details['Type'] = re.findall(type_pattern, div_soup)[0].strip()
    except:
        details['Type'] = -1
    
    try:
        details['Page'] = re.findall(page_pattern, div_soup)[0]  
    except:
        details['Page'] = -1
        
    try:
        details['Location'] = re.findall(location_pattern, div_soup)[0]
    except:
        details['Location'] = -1                                 
    
    return details

In [14]:
'''
get_page_location(div_soup) example


'''

soup_noteHeading = soup.find(attrs = {'class': 'noteHeading'})
print(soup_noteHeading)
print(get_page_location(soup_noteHeading.text.strip()))

<div class="noteHeading">
    Highlight(<span class="highlight_orange">orange</span>) - Page xxiv · Location 185
</div>
{'Type': 'Highlight(orange)', 'Page': 'xxiv', 'Location': '185'}


In [15]:
all_divs[6].text.strip()

'Highlight(orange) - Page xxiv · Location 185'

In [16]:
get_page_location(all_divs[22].text.strip())

{'Type': 'Highlight(yellow)', 'Page': '39', 'Location': '610'}

In [17]:
soup.find('div').find_next()

<div class="notebookFor">
                Notebook Export
            </div>

In [37]:
all_divs = soup.findAll('div')
section = 0

section_headers = {}
section_texts = pd.DataFrame()

current_type = None
current_page = None
current_location = None

for idx,div_tag in enumerate(all_divs):

    if div_tag['class'][0] == 'sectionHeading':

        section += 1
        section_headers[section] = div_tag.text.strip()
        
    
    if div_tag['class'][0] == 'noteHeading':
        div_text = div_tag.text.strip() #get text of div
        page_location = get_page_location(div_text) #extract page & location
        
        current_type = page_location['Type']
        current_page = page_location['Page']
        current_location = page_location['Location']
        
        if current_type == 'Bookmark':
            text = pd.Series([section, current_type, '', current_page, current_location],
                index = ['Section', 'Type', 'Text', 'Page', 'Location'])
            section_texts = pd.concat([section_texts, text], axis = 1)
        
        
    if div_tag['class'][0] == 'noteText':
        text = pd.Series([idx, section, current_type, div_tag.text.strip(), current_page, current_location],
                        index = ['div_index', 'Section', 'Type', 'Text', 'Page', 'Location'])
        section_texts = pd.concat([section_texts, text], axis = 1)




/Users/huangheqing/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/Users/huangheqing/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [38]:
section_headers

{1: 'THE PAINFUL PROLOGUE',
 2: 'INTRODUCTION',
 3: 'PART I: ASPIRE',
 4: 'PART II: SUCCESS',
 5: 'PART III: FAILURE'}

In [39]:
section_texts.T

,Location,Page,Section,Text,Type,div_index
0,185,xxiv,1,We must begin by seeing ourselves and the worl...,Highlight(orange),7
0,210,2,2,The ego we see most commonly goes by a more ca...,Highlight(orange),10
0,238,4,2,"“If you start believing in your greatness, it ...",Highlight(yellow),12
0,266,6,2,Humble in our aspirations Gracious in our succ...,Highlight(yellow),14
0,457,26,3,So what is scarce and rare? Silence. The abili...,Highlight(yellow),17
0,486,28,3,The only relationship between work and chatter...,Highlight(yellow),19
0,569,35,3,Think about this the next time you face that c...,Highlight(yellow),21
0,610,39,3,mastery. The pretense of knowledge is our most...,Highlight(yellow),23
0,615,39,3,"Each fighter, to become great, he said, needs ...",Highlight(yellow),25
0,662,43,3,"It’s why the old proverb says, “When student i...",Highlight(yellow),27


In [40]:
get_page_location(all_divs[32].text.strip())

{'Type': 'Highlight(yellow)', 'Page': '49', 'Location': '728'}

In [116]:
# Define Markdown Style

markdown_title = '#'
markdown_author = "##"
markdown_section = "###"
markdown_location = "*"
markdown_text = ">"

In [83]:
content = section_texts.T
section_headers
#get_book_details(soup)

'THE PAINFUL PROLOGUE'

In [121]:
# Writing files 
title = get_book_details(soup)['Title']
author = get_book_details(soup)['Authors']
file_name = get_book_details(soup)['Title'] + ' notes.txt'

# Create and open file
notes = open(r"%s"%(file_name), "w")

# Writing metadata

notes.write("\n")
notes.write(markdown_title + ' ' + title + "\n")
notes.write(markdown_author + ' '+ author + "\n")
notes.write("\n --- \n")

# Start Writing
for index, row in content.iterrows():
    current_session = 1
    
    # write Section title at the beginning of each section
    # Lance Look here - what's wrong?
    
    if row['Section'] == current_session:
        
        notes.write(markdown_section + ' '+"Chapter %s: "%(current_session) + section_headers[current_session] +'\n')
        current_session += 1
        
    # write text
    
    notes.write(markdown_text + ' ' + row['Text'] )
    notes.write('  -- (%sLocation: %s  Page: %s%s)\n\n' % 
                (markdown_location, row['Location'], row['Page'], markdown_location) )


    
    
notes.close()

In [24]:
pd.DataFrame(np.vstack([a,b]))

,0,1,2,3
0,1,2,3,4
1,3,4,5,6


In [25]:
c = [12,3,4,5]

In [26]:
c.append(5)

In [27]:
c.append([5,6])

In [28]:
c.extend([5,6])

In [29]:
c

[12, 3, 4, 5, 5, [5, 6], 5, 6]

In [30]:
print(soup.prettify())

<!DOCTYPE html PUBLIC
"-//W3C//DTD XHTML 1.0 Strict//EN"
"XHTML1-s.dtd" >
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/TR/1999/REC-html-in-xml">
 <head>
  <meta charset="utf-8"/>
  <style>
   .bodyContainer {
            font-family: Arial, Helvetica, sans-serif;
            text-align: center;
            padding-left: 32px;
            padding-right: 32px;
        }
        
        .notebookFor {
            font-size: 18px;
            font-weight: 700;
            text-align: center;
            color: rgb(119, 119, 119);
            margin: 24px 0px 0px;
            padding: 0px;
        }
        
        .bookTitle {
            font-size: 24px;
            font-weight: 700;
            text-align: center;
            color: #333333;
            margin-top: 22px;
            padding: 0px;
        }
        
        .authors {
            font-size: 18px;
            font-weight: 700;
            text-align: center;
            color: rgb(119, 119, 119);
            ma

In [31]:
print(soup.body)

<body>
<div class="bodyContainer">
<div class="notebookFor">
                Notebook Export
            </div>
<div class="bookTitle">
                Ego Is the Enemy
            </div>
<div class="authors">
                Holiday, Ryan
            </div>
<div class="citation">
</div>
<hr/>
<div class="sectionHeading">
    THE PAINFUL PROLOGUE
</div><div class="noteHeading">
    Highlight(<span class="highlight_orange">orange</span>) - Page xxiv · Location 185
</div>
<div class="noteText">
    We must begin by seeing ourselves and the world in a new way for the first time. Then we must fight to be different and fight to stay different—that’s the hard part. I’m not saying you should repress or crush every ounce of ego in your life—or that doing so is even possible. These are just reminders, moral stories to encourage our better impulses.
</div><div class="sectionHeading">
    INTRODUCTION
</div><div class="noteHeading">
    Highlight(<span class="highlight_orange">orange</span>) - Pa

In [32]:
print(soup.find_all('div', class_='noteHeading'))

[<div class="noteHeading">
    Highlight(<span class="highlight_orange">orange</span>) - Page xxiv · Location 185
</div>, <div class="noteHeading">
    Highlight(<span class="highlight_orange">orange</span>) - Page 2 · Location 210
</div>, <div class="noteHeading">
    Highlight(<span class="highlight_yellow">yellow</span>) - Page 4 · Location 238
</div>, <div class="noteHeading">
    Highlight(<span class="highlight_yellow">yellow</span>) - Page 6 · Location 266
</div>, <div class="noteHeading">
    Highlight(<span class="highlight_yellow">yellow</span>) - TALK, TALK, TALK &gt; Page 26 · Location 457
</div>, <div class="noteHeading">
    Highlight(<span class="highlight_yellow">yellow</span>) - TALK, TALK, TALK &gt; Page 28 · Location 486
</div>, <div class="noteHeading">
    Highlight(<span class="highlight_yellow">yellow</span>) - TO BE OR TO DO? &gt; Page 35 · Location 569
</div>, <div class="noteHeading">
    Highlight(<span class="highlight_yellow">yellow</span>) - BECOME A STUDE

In [33]:
s_tag = soup.span

In [34]:
s_tag.decomposed()

TypeError: 'NoneType' object is not callable

In [ ]:
print(soup.body)

In [ ]:
x = 0

div_tag = soup.find('div')
section = 0

section_headers = pd.DataFrame()
section_texts = pd.DataFrame()

current_type = None
current_page = None
current_location = None

while (x < 10):
    
    print(div_tag['class'][0])
    
    if div_tag['class'][0] == 'sectionHeading':
        print('hi')

        section += 1
        header = pd.Series([section, div_tag.text.strip()], index = ['Section', 'Text'])
        
        section_headers = pd.concat([section_headers, header], axis = 1)
        
        try:
            print('stop')
            div_tag = div_tag.find_next()
        except:
            print('stop')
            break
    
    if div_tag['class'][0] == 'noteHeading':
        
        div_text = div.text.strip() #get text of div
        page_location = get_page_location(div_text) #extract page & location
        
        current_type = page_location['Type']
        current_page = page_location['Page']
        current_location = page_location['Location']
        
        try:
            div_tag = div_tag.find_next()
        except:
            print('stop')
            break
        
    if div_tag['class'][0] == 'noteText':
        text = pd.Series([current_type, div_tag.text.strip(), current_page, current_location],
                        index = ['Type', 'Text', 'Page', 'Location'])
        section_texts = pd.concat([section_texts, text], axis = 1)
        
        try:
            div_tag = div_tag.find_next()
        except:
            print('stop')
            break
    
    try:
        div_tag = div_tag.find_next()
    except:
        break
            

    x+=1
